In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

fatal: destination path 'LightGBM' already exists and is not an empty directory.
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "2.2") 
-- OpenCL include directory:

In [2]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import random
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.cluster import KMeans

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = '/content/drive/Shareddrives/dacon/404/preprocessed/'
trainset = pd.read_csv(path + 'stack_train.csv', index_col=['Unnamed: 0'])
testset = pd.read_csv(path + 'stack_test.csv', index_col=['Unnamed: 0'])

In [4]:
trainset

,0_t,1_t,2_t,3_t,4_t,5_t,6_t,7_t,8_t,9_t,10_t,11_t,12_t,13_t,14_t,15_t,16_t,17_t,18_t,19_t,20_t,21_t,22_t,23_t,24_t,25_t,26_t,27_t,28_t,29_t,30_t,31_t,32_t,33_t,34_t,35_t,36_t,37_t,38_t,39_t,...,20201127000000,20201127060000,20201127120000,20201127180000,20201128000000,20201128060000,20201128120000,20201128180000,20201129000000,20201129060000,20201129120000,20201129180000,20201130000000,20201130060000,20201130120000,20201130180000,20201201000000,20201201060000,20201201120000,20201201180000,20201202000000,20201202060000,20201202120000,20201202180000,Group_0,Group_1,Group_2,Group_3,Group_4,Group_5,Group_6,model_multi,model_one,LGBM_0,LGBM_1,CAT_0,CAT_1,GBC_0,GBC_1,TARGET
0,-0.461538,-1.0,-1.0,-0.461538,-1.000000,-1.0000,-0.388889,1.187500,0.265255,-0.461538,-4.0,-1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.183844,0.194546,-0.384615,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,12.0,0.0,-0.038365,0.0,-0.384615,...,0.315465,-0.315465,-0.630930,0.101557,0.369070,-0.315465,-0.630930,0.417022,0.315465,0.254692,-0.630930,-0.630930,0.101557,-0.315465,-0.630930,0.101557,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,1,0,0,0,0,0,0,0,1,0.871132,0.128868,0.710593,0.289407,0.851778,0.148222,0
1,-3.000000,-3.0,-3.0,-3.000000,-3.000000,-3.0000,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.000000,-3.0,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.00,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.000000,-3.0,-3.000000,...,-0.630930,0.796095,2.433031,2.275097,0.254692,0.417022,0.315465,0.658521,0.709142,0.709142,0.684535,0.417022,0.500000,0.101557,0.184535,0.369070,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,0,1,0,0,0,0,0,1,0.097220,0.902780,0.054967,0.945033,0.127434,0.872566,1
2,-0.307692,0.0,0.0,-0.230769,0.166667,0.3849,0.277778,0.205357,0.052898,-0.230769,8.0,4.618802,2.0,4.666667,8.150555,0.0,0.0,0.0,12.0,6.0,3.0,1.25,0.039395,0.176151,-0.230769,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,-1.0,24.0,1.2,0.594091,1.2,-0.230769,...,0.315465,0.184535,-0.630930,0.101557,-0.630930,-0.630930,-0.630930,-0.630930,-0.130930,0.101557,-0.630930,-0.130930,-0.315465,0.101557,-0.315465,0.000000,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,1,0,0,0,0,0,0,1,0.781164,0.218836,0.638828,0.361172,0.826242,0.173758,0
3,-3.000000,-3.0,-3.0,-3.000000,-3.000000,-3.0000,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.000000,-3.0,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.00,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.000000,-3.0,-3.000000,...,0.369070,0.417022,-0.630930,0.101557,0.369070,0.000000,-0.630930,-0.630930,0.536429,-0.630930,-0.130930,-0.630930,-0.130930,-0.315465,-0.630930,0.000000,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,1,0,0,0,0,0,0,1,0.926358,0.073642,0.832738,0.167262,0.897152,0.102848,0
4,-0.307692,0.0,0.0,-0.307692,0.000000,0.0000,-0.222222,-0.212500,-0.106568,-0.230769,8.0,4.618802,1.0,12.666667,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.183844,0.030973,-0.230769,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.0,0.0,-1.0,0.0,0.0,0.329180,0.0,-0.230769,...,0.500000,0.184535,0.101557,0.101557,-0.130930,0.254692,0.315465,0.000000,0.460399,0.369070,-0.130930,0.000000,-0.130930,0.254692,0.101557,0.460399,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,0,0,0,1,0,0,0,1,0.479962,0.520038,0.457581,0.542419,0.511012,0.488988,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.307692,0.0,0.0,0.307692,0.000000,0.0000,0.166667,-0.416667,-0.097236,0.307692,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.137883,-0.050813,0

In [5]:
testset

,0_t,1_t,2_t,3_t,4_t,5_t,6_t,7_t,8_t,9_t,10_t,11_t,12_t,13_t,14_t,15_t,16_t,17_t,18_t,19_t,20_t,21_t,22_t,23_t,24_t,25_t,26_t,27_t,28_t,29_t,30_t,31_t,32_t,33_t,34_t,35_t,36_t,37_t,38_t,39_t,...,20201126180000,20201127000000,20201127060000,20201127120000,20201127180000,20201128000000,20201128060000,20201128120000,20201128180000,20201129000000,20201129060000,20201129120000,20201129180000,20201130000000,20201130060000,20201130120000,20201130180000,20201201000000,20201201060000,20201201120000,20201201180000,20201202000000,20201202060000,20201202120000,20201202180000,Group_0,Group_1,Group_2,Group_3,Group_4,Group_5,Group_6,model_multi,model_one,LGBM_0,LGBM_1,CAT_0,CAT_1,GBC_0,GBC_1
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30000,-0.461538,-1.0,-1.0,-0.461538,-1.0,-1.0,-0.444444,0.187500,-1.192347,-0.384615,6.00,-1.000000,1.0,1.666667,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.183844,-0.132599,-0.384615,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-0.2,-0.170820,-0.2,-0.384615,...,1.655537,-0.315465,0.754692,-0.630930,0.101557,-0.130930,0.000000,0.000000,0.184535,0.536429,-0.630930,0.254692,0.000000,-0.630930,-0.630930,-0.630930,0.184535,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,0,0,0,1,0,0,1,0,0.109736,0.890264,0.016204,0.983796,0.051095,0.948905
30001,0.230769,0.0,0.0,0.230769,0.0,0.0,0.388889,26.703125,73.453272,0.230769,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,65.111421,162.663537,0.307692,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,-1.0,0.0,0.2,0.353875,0.2,0.307692,...,-0.630930,-0.130930,-0.630930,-0.630930,-0.630930,-0.630930,-0.630930,-0.630930,-0.630930,0.184535,-0.630930,-0.630930,-0.630930,-0.630930,-0.630930,-0.630930,0.101557,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,1,0,0,0,0,0,0,0,1,0.783375,0.216625,0.609788,0.390212,0.623300,0.376700
30002,0.692308,0.0,0.0,0.692308,0.0,0.0,1.055556,0.705357,1.909760,0.692308,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.00,0.836490,2.221289,0.769231,0.0,0.0,0.0,0.0,0.0,12.0,5.0,2.0,1.0,-1.0,0.0,1.6,1.737499,1.6,0.769231,...,1.149509,0.536429,0.254692,-0.315465,0.460399,0.754692,0.417022,0.500000,-0.130930,0.500000,0.500000,0.254692,-0.130930,0.536429,0.536429,0.500000,0.254692,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,0,0,0,0,1,0,0,1,0.488154,0.511846,0.375431,0.624569,0.464438,0.535562
30003,-0.461538,-1.0,-1.0,-0.461538,-1.0,-1.0,-0.111111,0.116071,-0.163823,-0.076923,16.80,3.286335,3.0,12.000000,6.259006,0.0,0.0,0.0,0.0,0.0,0.0,0.50,-0.091922,0.210679,-0.384615,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,3.0,1.0,0.0,0.6,0.957062,0.6,-0.384615,...,-0.315465,-0.630930,-0.130930,-0.630930,-0.315465,0.254692,-0.315465,0.000000,0.254692,-0.630930,0.369070,-0.130930,-0.630930,-0.630930,0.460399,0.101557,0.417022,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,1,0,0,0,0,0,0,1,0.162863,0.837137,0.054893,0.945107,0.167469,0.832531
30004,0.000000,0.0,0.0,0.000000,0.0,0.0,-0.111111,-0.455357,-0.158522,0.153846,3.75,2.815772,2.0,2.916667,2.872764,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.183844,-0.132599,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,-0.038365,0.0,0.076923,...,0.417022,0.570157,0.417022,0.184535,0.834044,0.000000,0.000000,0.315465,0.973986,0.684535,-0.630930,0.101557,0.834044,0.369070,0.369070,0.101557,0.101557,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,0,0,0,0,1,0,1,0,0.098861,0.901139,0.031869,0.968131,0.123051,0.876949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,0.384615,0.0,0.0,0.384615,0.0,0.0,0.166667,-0.500000,-0.225512,0.384615,0.00,0.000000,0.0,0.000000,0.00

In [6]:
X = trainset.iloc[:,:-1]
y = trainset.iloc[:,-1]

In [7]:
train_X, valid_X, train_y,  valid_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
fit_params={"eval_set" : [(valid_X, valid_y)], "eval_metric" : 'auc', 'verbose': 100}

from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

param_test ={'num_leaves': [5,10,20,40], 
             'min_child_samples': [10,40,80,150,250,400,600], 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': [0.4,0.6,0.8,1], 
             'colsample_bytree': [0.4,0.6,0.8,1],
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
             'n_estimators' : [500,750,1000,1250,1500,2000,3000],
             'min_split_gain' : [0, 0.01, 0.1, 1,10,100],
             'feature_fraction' : [0.6, 0.8, 1],
             'max_depth' : [5, 10,15,20]}


gnum_leaves = [5,10,20,40]
gmin_child_samples= [10,40,80,150,250,400,600]
gmin_child_weight= [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4]
gsubsample= [0.4,0.6,0.8,1]
gcolsample_bytree= [0.4,0.6,0.8,1]
# greg_alpha= [0, 1e-1, 1, 2, 5, 7, 10, 50, 100]
# greg_lambda= [0, 1e-1, 1, 5, 10, 20, 50, 100]
gn_estimators = [500,750,1000,1250,1500,2000,3000]
gmin_split_gain = [0, 0.01, 0.1, 1,10,100]
gfeature_fraction = [0.6, 0.8, 1]
gmax_depth = [5, 10,15,20]

In [ ]:
ini_scores = 0.5

for num in tqdm(gnum_leaves):
  for min_sam in gmin_child_samples:
    for min_wei in gmin_child_weight:
      for sub in gsubsample:
        for n_e in gn_estimators:
          for min_spli in gmin_split_gain:
            for gf in gfeature_fraction:
              for gma in gmax_depth:
                param_test ={'num_leaves': num, 
                'min_child_samples': min_sam, 
                'min_child_weight': min_wei,
                'subsample': sub, 
                'n_estimators' : n_e,
                'min_split_gain' : min_spli,
                'feature_fraction' : gf,
                'max_depth' : gma}


                clf = LGBMClassifier(**param_test, device='gpu', is_unbalance = True, random_state=42).fit(X,y)
                y_pred = cross_val_predict(clf, X, y, cv=5, method='predict_proba')
                ans = roc_auc_score(y, y_pred[:, 1])
                if ans > ini_scores:
                  ini_scores = ans
                  with open("/content/drive/MyDrive/stack_params.txt", "a") as file:
                    file.write("num_leaves : {0} / min_child_samples : {1} / min_child_weight : {2} /subsample : {3} / n_estimators : {4}  / min_split_gain : {5}  / feature_fraction : {6}  / max_depth : {7} / SCORE : {8}\n".format(num, min_sam, min_wei, sub, n_e,min_spli, gf, gma, ans))
                    file.close()

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGB

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = LGBMClassifier(device='gpu', is_unbalance = True, random_state=42)
rs = GridSearchCV(estimator=clf, param_grid=param_test, scoring='roc_auc', cv=5, refit=True, verbose=3)

rs.fit(train_X, train_y, **fit_params)
print('Best score reached: {} with params: {} '.format(rs.best_score_, rs.best_params_))